In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from typing import Type
from torch import nn
from torch.optim import optimizer
import rasterio
import zipfile
from matplotlib import pyplot as plt
import datetime
from torchvision import transforms as transforms
import shutil
import torchmetrics
import os
import pytorch_lightning as pl
import sklearn
from torch.nn import functional as F
import tqdm

# --- GPU selection --- #
gpus = 7 # slot number (e.g., 3), no gpu use -> write just ' '
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpus)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [2]:
def minmax(array : Type[np.ndarray], dim = 0):
    min = np.min(array, axis=dim)
    max = np.max(array, axis=dim)
    array = (array-min)/(max-min)
    return array

def log_minmax(array : Type[np.ndarray], dim = 0):
    min = array.min()
    array = array - min + 1
    array = np.log(array)
    max = array.max()
    array = (array)/(max)
    return array

In [3]:
def save_result(model: Type[nn.Module], dataloader : Type[DataLoader], path:str, description:str = '', reference_data:str = '', patch_size:int = 60, now = datetime.datetime.now()):
    best_model = model
    os.makedirs(os.path.join(path,f'{now.year}.{now.month}.{now.day}/', f'{description}/','tmp/'), exist_ok=True)
    zipped_results = zipfile.ZipFile(os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/','RESULT_{0:0=2d}:{1:0=2d}'.format(now.hour, now.minute)+f'_{description}.zip'), 'w')
    prediction = np.zeros((2400//patch_size,2400//patch_size,7))

    with tqdm.tqdm(enumerate(dataloader)) as data_pbar:
        data_pbar.set_description('Predicting...')
        for i, (data, index_OHE, index) in data_pbar:
            prediction[i, :, :] = best_model(data).detach().numpy()

    

    prediction_expanded = np.zeros((7,2400,2400))
    for i in range(2400//patch_size):
        for j in range(2400//patch_size):
            for k in range(7):
                prediction_expanded[k,i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size] = prediction[i,j,k]

    reference_image = rasterio.open(reference_data)
    layer_index = [1,2,7,8,9,10,11]

    with tqdm.trange(prediction_expanded.shape[0]) as write_pbar:
        write_pbar.set_description('Writing data')
        for i in write_pbar:
            #print('a') 
            processed_tiff = rasterio.open(
                os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/', 'tmp/', f'Result_{layer_index[i]}_{description}.tif'),
                'w',
                driver='GTiff',
                height=prediction_expanded.shape[1],
                width=prediction_expanded.shape[2],
                count=1,
                dtype=prediction_expanded.dtype,
                crs=reference_image.crs,
                transform=reference_image.transform,
            )
            #print('b')
            processed_tiff.write(prediction_expanded[i,:,:],1)
            processed_tiff.close()
            #print('c')
            zipped_results.write(os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/', 'tmp/', f'Result_{layer_index[i]}_{description}.tif'), f'Result_{layer_index[i]}_{description}.tif')

    zipped_results.close()
    return os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/','RESULT_{0:0=2d}:{1:0=2d}'.format(now.hour, now.minute)+f'_{description}.zip')


In [4]:
def train_model(model, dataloaders, criterion, optimizer, scheduler, device, num_epochs=13, train_rate: float = 0.8, batch_size: int = 60, path:str = '../Data/N12/Model/', description:str = 'no_description', reference_data:str = ''): 
    train_loss_history = []
    valid_loss_history = []

    patch_size = dataloaders['Train'].dataset.data.shape[-1]
    training_patches = len(dataloaders['Train'].dataset)
    validating_patches = len(dataloaders['Validation'].dataset)
    print(f'Training Patches : {training_patches}\nValidating Patches : {validating_patches}')

    best_model_epoch = 0
    least_valid_loss = 100
    now = datetime.datetime.now()
    os.makedirs(os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/', 'tmp/'), exist_ok=True)
    zipped_model = zipfile.ZipFile(os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/', '{0:0=2d}:{1:0=2d}'.format(now.hour, now.minute)+f'_{description}'+'.zip'), 'w')
    
    epoch_range = tqdm.trange(num_epochs)
    for epoch in epoch_range:

        train_running_loss = 0.0
        valid_running_loss = 0.0

        epoch_range.set_description(f'EPOCH #{epoch}')
        

        for state in ['Train', 'Validation']:
            #pbar = tqdm.tqdm(dataloaders[state])
            for inputs, labels_OHE, labels in dataloaders[state]:
                
                inputs = inputs.to(device)
                labels = labels.to(device)
                model.to(device)
                
                outputs = model(inputs)
                
                optimizer.zero_grad()

                if state == 'Train':
                    model.train()
                    train_loss = criterion(outputs, labels)
                    train_loss.backward()
                    train_running_loss += train_loss.item() * inputs.size(0)
                    #pbar.set_description('Train')
                
                if state == 'Validation':
                    model.eval()
                    valid_loss = criterion(outputs, labels)
                    valid_running_loss += valid_loss.item() * inputs.size(0)
                    #pbar.set_description('Valid')

                optimizer.step()
                #valid_running_similarity += metric(outputs, labels)
                #print('validating')
                
                #print(f'{i}th batch')
            #pbar.clear()
            
        epoch_range.refresh()

        
        #print(f'Memory after a training : {torch.cuda.memory_allocated()/1024/1024}')

        epoch_train_loss = train_running_loss / training_patches
        epoch_valid_loss = valid_running_loss / validating_patches
        scheduler.step(epoch_valid_loss)

        #print(f'Valid loss: {epoch_valid_loss} | Train loss: {epoch_train_loss}')


        if epoch_valid_loss < least_valid_loss:
            least_valid_loss = epoch_valid_loss
            best_model_epoch = epoch

        train_loss_history.append(epoch_train_loss)      
        valid_loss_history.append(epoch_valid_loss)

        torch.save(model.state_dict(), os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/','tmp/', '{0:0=2d}.pth'.format(epoch)))
        zipped_model.write(os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/','tmp/', '{0:0=2d}.pth'.format(epoch)))

    plt.figure(figsize=(20,8))
    plt.plot(train_loss_history, 'r-')
    plt.plot(valid_loss_history, 'bo')
    plt.savefig(os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/','tmp/', 'Tendency.png'), dpi=300)
    zipped_model.write(os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/','tmp/', 'Tendency.png'))
    zipped_model.writestr('README.txt', f'{description}\nThe best Model : #{best_model_epoch}th model with loss {least_valid_loss}\nOptimizer : {optimizer}\nLoss function : {criterion}\nBatch size : {batch_size}\nScheduler : {scheduler}\nPatch size : {patch_size}\nTotal epochs : {num_epochs}\nModel information :\n{model.modules}')
    
    print('Best loss: {:4f}, in Epoch #{:0=3d}'.format(least_valid_loss, best_model_epoch))    
    zipped_model.close()
    shutil.copy(src=os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/', 'tmp/', '{0:0=2d}.pth'.format(epoch)), dst=os.path.join(path,f'{now.year}.{now.month}.{now.day}/', 'Best_Model_Parameters_of_{0:0=2d}:{1:0=2d}'.format(now.hour, now.minute)+f'_{description}'+'.pth'))
    print('Model information is saved in '+os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/', '{0:0=2d}:{1:0=2d}'.format(now.hour, now.minute)+f'_{description}'+'.zip'))

    model.load_state_dict(torch.load(os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/','tmp/', '{0:0=2d}.pth'.format(best_model_epoch))))
    result_path = save_result(model = model.to('cpu'), dataloader=dataloaders['Prediction'], path=path, description=description, reference_data=reference_data, patch_size=patch_size, now=now)
    print('Model result is saved in '+ result_path)
    
    shutil.rmtree(os.path.join(path,f'{now.year}.{now.month}.{now.day}/',f'{description}/','tmp/'))
    best_model_path = os.path.join(path,f'{now.year}.{now.month}.{now.day}/', 'Best_Model_Parameters_of_{0:0=2d}:{1:0=2d}'.format(now.hour, now.minute)+f'_{description}'+'.pth')
    return best_model_path

In [5]:
def prepare_raw_files(region:str):
    if (os.path.exists(f'../Data/{region}/np/train_array.npy') and os.path.exists(f'../Data/{region}/np/target_array_OHE.npy')) and (os.path.exists(f'../Data/{region}/np/target_array_RAW.npy')):
        print('Preexisting Data Found. Loading...')
        train_array = np.load(f'../Data/{region}/np/train_array.npy')
        target_array_OHE = np.load(f'../Data/{region}/np/target_array_OHE.npy')
        target_array = np.load(f'../Data/{region}/np/target_array_RAW.npy')
    else:
        print('No Data Found. Loading from Raw Data')
        lidar_image = rasterio.open(f'../Data/{region}/{region}_lidar.tif').read()
        lidar_array = np.array(lidar_image)
        lidar_array = log_minmax(lidar_array, dim=(0,1))

        lidar_1n_image = rasterio.open(f'../Data/{region}/{region}_lidar_1n.tif').read()
        lidar_1n_array = np.array(lidar_1n_image)
        lidar_1n_array = log_minmax(lidar_1n_array, dim=(0,1))

        lidar_nt_image = rasterio.open(f'../Data/{region}/{region}_lidar_nt.tif').read()
        lidar_nt_array = np.array(lidar_nt_image)
        lidar_nt_array = log_minmax(lidar_nt_array, dim=(0,1))

        RGB2020_image = rasterio.open(f'../Data/{region}/{region}_RGB2020.tif').read()
        RGB2020_array = np.array(RGB2020_image)

        train_array = np.stack([lidar_array, lidar_1n_array, lidar_nt_array]).squeeze()
        train_array = np.concatenate((train_array,RGB2020_array))
        target_image = rasterio.open(f'../Data/{region}/{region}_newlc.tif').read()
        target_array = np.array(target_image, dtype=int).squeeze()
        target_array = np.where(target_array == 1, 0, target_array)
        target_array = np.where(target_array == 2, 1, target_array)
        target_array = np.where(target_array == 7, 2, target_array)
        target_array = np.where(target_array == 8, 3, target_array)
        target_array = np.where(target_array == 9, 4, target_array)
        target_array = np.where(target_array == 10, 5, target_array)
        target_array = np.where(target_array == 11, 6, target_array)

        target_array_OHE = np.zeros(shape=(7,2400,2400))
        num = np.unique(target_array)

        num = max(num.shape[0],7)
        encoded_target_array = np.eye(num)[target_array]
        for i in range(encoded_target_array.shape[-1]):
            target_array_OHE[i,:,:]=encoded_target_array[:,:,i]
        
        os.makedirs(f'../Data/{region}/np', exist_ok=True)
        np.save(f'../Data/{region}/np/train_array.npy', train_array)
        np.save(f'../Data/{region}/np/target_array_RAW.npy', target_array)
        np.save(f'../Data/{region}/np/target_array_OHE.npy', target_array_OHE)

    return train_array, target_array.astype(int), target_array_OHE.astype(int)

In [6]:
class TrainDataset2(Dataset):
    def __init__(self, data_array : Type[np.ndarray], target_array_OHE : Type[np.ndarray], target_array_RAW : Type[np.ndarray], patch_size : int, is_evaluating : bool = False, is_validating : bool = False, rotate : bool = False, train_ratio : float = 0.8):
        self.is_validating = is_validating
        self.is_evaluating = is_evaluating
        seed = 386579

        #print(f'Data shape: {data_array.shape} | Target shape: {target_array.shape}')

        self.data = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size), data_array.shape[0], patch_size, patch_size))

        for i in range(0,data_array.shape[1]//patch_size):
            for j in range(0,data_array.shape[2]//patch_size):
                self.data[data_array.shape[1]//patch_size*i+j,:,:,:] = data_array[:,i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]

        self.label_OHE = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size), target_array_OHE.shape[0] ,patch_size, patch_size), dtype=float)
        for k in range(0,data_array.shape[1]//patch_size):
            for l in range(0,data_array.shape[2]//patch_size):
                self.label_OHE[data_array.shape[1]//patch_size*k+l,:,:,:] = target_array_OHE[:,i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]

        self.label_RAW = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size),data_array.shape[0]+1))
        for k in range(0,data_array.shape[1]//patch_size):
            for l in range(0,data_array.shape[2]//patch_size):
                self.label_RAW[data_array.shape[1]//patch_size*k+l,:] = np.bincount(target_array_RAW[k*patch_size:(k+1)*patch_size, l*patch_size:(l+1)*patch_size].reshape(-1), minlength=7)/(patch_size*patch_size)


        if not is_evaluating:
            if rotate:
                for i in range(2):
                    rotated_data = np.rot90(self.data, k=i+1, axes=(-2, -1))
                    self.data = np.concatenate((self.data, rotated_data), axis=0)
                    rotated_label_OHE = np.rot90(self.label_OHE, k=i+1, axes=(-2, -1))
                    rotated_label_RAW = self.label_RAW
                    self.label_OHE = np.concatenate((self.label_OHE, rotated_label_OHE), axis=0)
                    self.label_RAW = np.concatenate((self.label_RAW, rotated_label_RAW), axis=0)

        train_size = int(self.data.shape[0]*train_ratio)
        index_array = np.random.RandomState(seed=seed).permutation(self.data.shape[0])
        self.train_index = index_array[0:train_size]
        self.valid_index = index_array[train_size:index_array.shape[0]]
        
        self.data = torch.as_tensor(self.data).float()
        self.label_OHE = torch.as_tensor(self.label_OHE).float()
        self.label_RAW = torch.as_tensor(self.label_RAW).float()

        self.data[:,3:6,:,:] = self.data[:,3:6,:,:]/255

    def __len__(self):
        if self.is_evaluating:
            return self.data.shape[0]

        if self.is_validating:
            return self.valid_index.shape[0]
        else:
            return self.train_index.shape[0]

    def __getitem__(self, idx):
        if self.is_evaluating:
            sample = torch.as_tensor(self.data[idx,:,:,:]).float()
            label_OHE = torch.as_tensor(self.label_OHE[idx,:]).float()
            label_RAW = torch.as_tensor(self.label_RAW[idx,:]).float()
            return sample, label_OHE, label_RAW
        
        if self.is_validating:
            sample = torch.as_tensor(self.data[self.valid_index[idx],:,:,:]).float()
            label_OHE = torch.as_tensor(self.label_OHE[self.valid_index[idx],:]).float()
            label_RAW = torch.as_tensor(self.label_RAW[self.valid_index[idx],:]).float()
        else:
            sample = torch.as_tensor(self.data[self.train_index[idx],:,:,:]).float()
            label_OHE = torch.as_tensor(self.label_OHE[self.train_index[idx],:]).float()
            label_RAW = torch.as_tensor(self.label_RAW[self.train_index[idx],:]).float()

        return sample, label_OHE, label_RAW

In [7]:
def fixed_padding(inputs, kernel_size, dilation):
    kernel_size_effective = kernel_size + (kernel_size - 1) * (dilation - 1)
    pad_total = kernel_size_effective - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    padded_inputs = F.pad(inputs, (pad_beg, pad_end, pad_beg, pad_end))
    return padded_inputs


class SeparableConv2d(nn.Module):
    def __init__(self, inplanes, planes, kernel_size=3, stride=1, dilation=1, bias=False, BatchNorm=None):
        super(SeparableConv2d, self).__init__()

        self.conv1 = nn.Conv2d(inplanes, inplanes, kernel_size, stride, 0, dilation,
                               groups=inplanes, bias=bias)
        self.bn = BatchNorm(inplanes)
        self.pointwise = nn.Conv2d(inplanes, planes, 1, 1, 0, 1, 1, bias=bias)

    def forward(self, x):
        x = fixed_padding(x, self.conv1.kernel_size[0], dilation=self.conv1.dilation[0])
        x = self.conv1(x)
        x = self.bn(x)
        x = self.pointwise(x)
        return x

In [8]:
class ConvBN(nn.Module):
    def __init__(self, C_in, C_out, kernel_size, stride, dilation = 1, affine=True, fix_padding = False):
        super(ConvBN, self).__init__()
        self.fix_padding = fix_padding
        self.conv2d = nn.Conv2d(C_in, C_in, kernel_size=kernel_size, stride=stride, groups=C_in, bias=False, dilation=dilation)
        self.pointwise = nn.Conv2d(C_in, C_out, kernel_size=1, padding=0, bias=False)
        self.batchnorm = nn.BatchNorm2d(C_in, affine=affine)
        

    def forward(self, x):
        if self.fix_padding:
           x = fixed_padding(x, self.conv2d.kernel_size[0], dilation=self.conv2d.dilation[0])
        x = self.conv2d(x)
        x = self.batchnorm(x)
        x = self.pointwise(x)
        return x

In [9]:
class UrbanGreenRegression(nn.Module):
    def __init__(self):
        super(UrbanGreenRegression, self).__init__()
        '''self.conv_block_1 = nn.Sequential(
            ConvBN(6,32,3,1),#98
            nn.ReLU(),
            ConvBN(32,32,3,1),#96
            nn.ReLU(),
            ConvBN(32,32,3,1),#94
            nn.ReLU(),
            nn.MaxPool2d(2)#47
        )
        self.conv_block_2 = nn.Sequential(
            ConvBN(32,64,3,1),#45
            nn.ReLU(),
            ConvBN(64,64,3,1),#43
            nn.ReLU(),
            ConvBN(64,64,3,1),#41
            nn.ReLU(),
            nn.MaxPool2d(2)#8
        )'''

        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(6,1,kernel_size=1,stride=1),
            nn.BatchNorm2d(1),
            nn.ReLU()
        )

        self.fc_block_1 = nn.Sequential(
            nn.Linear(in_features=100, out_features=256, bias=False),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=256, bias=False),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=256, bias=False),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU(),
            nn.Linear(in_features=256, out_features=256, bias=False),
            nn.BatchNorm1d(num_features=256),
            nn.ReLU()
        )

        self.fc_block_2 = nn.Sequential(
            nn.Linear(in_features=256, out_features=64, bias=False),
            nn.BatchNorm1d(num_features=64),
            nn.ReLU(),
            nn.Linear(in_features=64, out_features=64, bias=False),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.Linear(64,7, False),
            nn.BatchNorm1d(7)
        )

    def forward(self,x):
        x = self.conv_block_1(x)
        #x = self.conv_block_2(x)
        #print(x.shape)
        x = x.view(x.shape[0], -1)
        #print(x.shape)
        x = self.fc_block_1(x)
        x = self.fc_block_2(x)
        return torch.softmax(x, dim=-1)

In [11]:
raw_data_array ,raw_target_array, OHE_target_array = prepare_raw_files('N12')
batch_size = 4
patch_size = 100
train_ratio = 0.8
rotate_training_data = False
Datasets_NON_OHE = {
    'Train' : TrainDataset2(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, rotate = rotate_training_data, train_ratio = train_ratio),
    'Validation' : TrainDataset2(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, is_validating = True, rotate = rotate_training_data, train_ratio = train_ratio),
    'Prediction' : TrainDataset2(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, is_evaluating = True, train_ratio = train_ratio)
}

No Data Found. Loading from Raw Data


In [11]:
Dataloaders_NON_OHE = {
    'Train' : DataLoader(Datasets_NON_OHE['Train'], batch_size=batch_size),
    'Validation' : DataLoader(Datasets_NON_OHE['Validation'], batch_size=batch_size),
    'Prediction' : DataLoader(Datasets_NON_OHE['Prediction'], batch_size=2400//patch_size)
}

NameError: name 'Datasets_NON_OHE' is not defined

In [29]:
from torchsummary import summary
model_for_summary = UrbanGreenRegression()
model_for_summary.to(device)
summary(model_for_summary, input_size=(6,10,10), batch_size=64)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [64, 1, 10, 10]               7
       BatchNorm2d-2            [64, 1, 10, 10]               2
              ReLU-3            [64, 1, 10, 10]               0
            Linear-4                  [64, 256]          25,600
       BatchNorm1d-5                  [64, 256]             512
              ReLU-6                  [64, 256]               0
            Linear-7                  [64, 256]          65,536
       BatchNorm1d-8                  [64, 256]             512
              ReLU-9                  [64, 256]               0
           Linear-10                  [64, 256]          65,536
      BatchNorm1d-11                  [64, 256]             512
             ReLU-12                  [64, 256]               0
           Linear-13                  [64, 256]          65,536
      BatchNorm1d-14                  [

In [ ]:
model2 = UrbanGreenRegression()
criterion2 = torch.nn.MSELoss(reduction='mean')
optimizer2 = torch.optim.SGD(model2.parameters(), lr=0.01, momentum=0.9)
scheduler2 = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer2, 'min', patience=5, factor=0.75)
best_model_path = train_model(model2, Dataloaders_NON_OHE, criterion2, optimizer2, scheduler2, device, num_epochs=100, batch_size=batch_size, path='/home/bcyoon/Byeongchan/Data/N12/Model/Segmentation/Regression/', description='patch_10_pointwiseConv')

In [ ]:
best_model_path = '/home/bcyoon/Byeongchan/Data/N12/Model/Segmentation/Regression/2022.7.26/Best_Model_Parameters_of_15:32_patch_10_pointwiseConv.pth'
model2.load_state_dict(torch.load(best_model_path))
result_path = save_result(model = model2.to('cpu'), dataloader=Dataloaders_NON_OHE['Prediction'], path='/home/bcyoon/Byeongchan/Data/N12/Model/Segmentation/Regression/', description='patch_10_pointwiseConv', reference_data='/home/bcyoon/Byeongchan/Data/N12/N12_lidar.tif', patch_size=patch_size, now=datetime.datetime.now())

In [10]:
class ConvBN(nn.Module):
    def __init__(self, Cin, Cout, kernel_size, stride=1):
        super(ConvBN,self).__init__()
        self.conv = nn.Conv2d(Cin, Cin, kernel_size=kernel_size, stride=stride, groups=Cin, bias=False)
        self.batchnorm = nn.BatchNorm2d(Cin)
        self.pointwise = nn.Conv2d(Cin, Cout, kernel_size=1, padding=0, bias=False)
    def forward(self,x):
        x = self.conv(x)
        x = self.batchnorm(x)
        x = self.pointwise(x)
        return x

In [11]:
class SegBlock(nn.Module):
    def __init__(self, skip:bool, in_channel:int, out_channel:int, is_last:bool = False):
        super(SegBlock,self).__init__()
        self.skip = skip
        self.is_last = is_last
        self.conv1 = ConvBN(in_channel, out_channel, kernel_size=3, stride=1)
        self.relu1 = nn.ReLU()
        self.conv2 = ConvBN(out_channel, out_channel, kernel_size=3, stride=1)
        self.relu2 = nn.ReLU()

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        return x

In [12]:
def crop_add(skip:Type[torch.Tensor], target:Type[torch.Tensor])->torch.Tensor:
    cropped_skip = skip[:,:,(skip.shape[-2]-target.shape[-2])//2:(skip.shape[-2]+target.shape[-2])//2,(skip.shape[-1]-target.shape[-1])//2:(skip.shape[-1]+target.shape[-1])//2]
    return torch.cat((cropped_skip, target), dim=1)

In [13]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.enc1 = SegBlock(True, 3,64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = SegBlock(True, 64,128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = SegBlock(True, 128, 256)
        self.pool3 = nn.MaxPool2d(2)
        self.enc4 = SegBlock(True, 256, 512)
        self.pool4 = nn.MaxPool2d(2)
        self.dec0 = SegBlock(False, 512, 1024)
        self.upconv1 = nn.ConvTranspose2d(1024, 512, kernel_size=2, stride=2)
        self.dec1 = SegBlock(False,1024,512)
        self.upconv2 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec2 = SegBlock(False,512,256)
        self.upconv3 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec3 = SegBlock(False,256,128)
        self.upconv4 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec4 = SegBlock(False,128,64, is_last = True)

    def forward(self,x):
        enc1 = self.enc1(x)
        x = self.pool1(enc1)
        enc2 = self.enc2(x)
        x = self.pool2(enc2)
        enc3 = self.enc3(x)
        x = self.pool3(enc3)
        enc4 = self.enc4(x)
        x = self.pool4(enc4)
        x = self.dec0(x)
        x = self.upconv1(x)
        x = self.dec1(crop_add(enc4, x))
        x = self.upconv2(x)
        x = self.dec2(crop_add(enc3, x))
        x = self.upconv3(x)
        x = self.dec3(crop_add(enc2, x))
        x = self.upconv4(x)
        x = self.dec4(crop_add(enc1, x))
        return x


In [14]:
def mirror_extrapolate(inp:torch.Tensor, in_shape:tuple = (100,100), out_shape:tuple = (284,284)) -> torch.Tensor: # 배치 포함해서 input으로 넣어야 함
    pad = ((out_shape[0]-in_shape[0])//2,(out_shape[0]-in_shape[0])//2,(out_shape[0]-in_shape[0])//2,(out_shape[0]-in_shape[0])//2)
    return F.pad(inp, pad=pad, mode="reflect")

In [15]:
class TrainDataset3(Dataset):
    def __init__(self, data_array : Type[np.ndarray], target_array_OHE : Type[np.ndarray], target_array_RAW : Type[np.ndarray], patch_size : int, is_evaluating : bool = False, is_validating : bool = False, rotate : bool = False, train_ratio : float = 0.8):
        self.is_validating = is_validating
        self.is_evaluating = is_evaluating
        seed = 386579

        #print(f'Data shape: {data_array.shape} | Target shape: {target_array.shape}')

        self.data = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size), data_array.shape[0], patch_size, patch_size))

        for i in range(0,data_array.shape[1]//patch_size):
            for j in range(0,data_array.shape[2]//patch_size):
                self.data[data_array.shape[1]//patch_size*i+j,:,:,:] = data_array[:,i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]

        self.label_OHE = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size), target_array_OHE.shape[0] ,patch_size, patch_size), dtype=float)
        for k in range(0,data_array.shape[1]//patch_size):
            for l in range(0,data_array.shape[2]//patch_size):
                self.label_OHE[data_array.shape[1]//patch_size*k+l,:,:,:] = target_array_OHE[:,i*patch_size:(i+1)*patch_size, j*patch_size:(j+1)*patch_size]

        self.label_RAW = np.zeros(((data_array.shape[1]//patch_size) * (data_array.shape[2]//patch_size),data_array.shape[0]+1))
        for k in range(0,data_array.shape[1]//patch_size):
            for l in range(0,data_array.shape[2]//patch_size):
                self.label_RAW[data_array.shape[1]//patch_size*k+l,:] = np.bincount(target_array_RAW[k*patch_size:(k+1)*patch_size, l*patch_size:(l+1)*patch_size].reshape(-1), minlength=7)/(patch_size*patch_size)


        if not is_evaluating:
            if rotate:
                for i in range(2):
                    rotated_data = np.rot90(self.data, k=i+1, axes=(-2, -1))
                    self.data = np.concatenate((self.data, rotated_data), axis=0)
                    rotated_label_OHE = np.rot90(self.label_OHE, k=i+1, axes=(-2, -1))
                    rotated_label_RAW = self.label_RAW
                    self.label_OHE = np.concatenate((self.label_OHE, rotated_label_OHE), axis=0)
                    self.label_RAW = np.concatenate((self.label_RAW, rotated_label_RAW), axis=0)

        train_size = int(self.data.shape[0]*train_ratio)
        index_array = np.random.RandomState(seed=seed).permutation(self.data.shape[0])
        self.train_index = index_array[0:train_size]
        self.valid_index = index_array[train_size:index_array.shape[0]]
        
        self.data = torch.as_tensor(self.data).float()
        self.data[:,3:6,:,:] = self.data[:,3:6,:,:]/255
        self.data_seg = mirror_extrapolate(self.data).squeeze()[:,0:3,:,:]
        self.data_reg = torch.zeros((self.data.shape[0], 100, 6, self.data.shape[-2]//10, self.data.shape[-1]//10))
        print(self.data.shape)
        for k in tqdm.trange(self.data.shape[0]):
            for i in range(self.data.shape[-1]//10):
                for j in range(self.data.shape[-2]//10):
                    self.data_reg[k,10*i+j,:,:,:] = self.data[k,:,10*j:10*j+10, 10*i:10*i+10]


        self.label_OHE = torch.as_tensor(self.label_OHE).float()
        self.label_RAW = torch.as_tensor(self.label_RAW).float()


    def __len__(self):
        if self.is_evaluating:
            return self.data.shape[0]

        if self.is_validating:
            return self.valid_index.shape[0]
        else:
            return self.train_index.shape[0]

    def __getitem__(self, idx):
        if self.is_evaluating:
            data_reg = torch.as_tensor(self.data_reg[idx,:,:,:]).float()
            data_seg = torch.as_tensor(self.data_seg[idx,:,:,:]).float()
            label_OHE = torch.as_tensor(self.label_OHE[idx,:]).float()
            label_RAW = torch.as_tensor(self.label_RAW[idx,:]).float()
            return data_seg, data_reg, label_OHE, label_RAW
        
        if self.is_validating:
            data_reg = torch.as_tensor(self.data_reg[self.valid_index[idx],:,:,:]).float()
            data_seg = torch.as_tensor(self.data_seg[self.valid_index[idx],:,:,:]).float()
            label_OHE = torch.as_tensor(self.label_OHE[self.valid_index[idx],:]).float()
            label_RAW = torch.as_tensor(self.label_RAW[self.valid_index[idx],:]).float()
        else:
            data_reg = torch.as_tensor(self.data_reg[self.train_index[idx],:,:,:]).float()
            data_seg = torch.as_tensor(self.data_seg[self.train_index[idx],:,:,:]).float()
            label_OHE = torch.as_tensor(self.label_OHE[self.train_index[idx],:]).float()
            label_RAW = torch.as_tensor(self.label_RAW[self.train_index[idx],:]).float()

        return data_seg, data_reg, label_OHE, label_RAW

In [16]:
raw_data_array ,raw_target_array, OHE_target_array = prepare_raw_files('N12')
batch_size = 4
patch_size = 100
train_ratio = 0.8
rotate_training_data = False
Datasets_ver3 = {
    'Train' : TrainDataset3(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, rotate = rotate_training_data, train_ratio = train_ratio),
    'Validation' : TrainDataset3(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, is_validating = True, rotate = rotate_training_data, train_ratio = train_ratio),
    'Prediction' : TrainDataset3(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, is_evaluating = True, train_ratio = train_ratio)
}
Dataloaders_ver3 = {
    'Train' : DataLoader(Datasets_ver3['Train'], batch_size=batch_size),
    'Validation' : DataLoader(Datasets_ver3['Validation'], batch_size=batch_size),
    'Prediction' : DataLoader(Datasets_ver3['Prediction'], batch_size=2400//patch_size)
}

Preexisting Data Found. Loading...
torch.Size([576, 6, 100, 100])


100%|██████████| 576/576 [00:01<00:00, 371.99it/s]


torch.Size([576, 6, 100, 100])


100%|██████████| 576/576 [00:01<00:00, 364.87it/s]


torch.Size([576, 6, 100, 100])


100%|██████████| 576/576 [00:02<00:00, 261.23it/s]


In [33]:
class Splitted_Regression(nn.Module):
    def __init__(self):
        super(Splitted_Regression, self).__init__()
        self.regression = UrbanGreenRegression() # 4차원 텐서만 받음.
        self.regression.load_state_dict(torch.load('/home/bcyoon/Byeongchan/Data/N12/Model/Segmentation/Regression/2022.7.26/Best_Model_Parameters_of_15:32_patch_10_pointwiseConv.pth'))
        self.regression.to('cuda:0')
        for param in self.regression.parameters():
            param.requires_grad = False
        #self.batchnorm = nn.BatchNorm2d(7)
    def forward(self, x):
        # x : (batch, 100, 6, 10, 10) 데이터
        out = torch.zeros(x.shape[0],7,100)
        out = out.to('cuda:0')
        for i in range(x.shape[0]):
            tmp = self.regression(x[i,:,:,:,:])
            #print(tmp.shape)
            out[i,:,:] = torch.transpose(tmp, 0, 1)
        out = out.view(x.shape[0], 7, 10, 10)
        #Bilinear Interpolation 추가할 것
        out = F.interpolate(out, size=(100,100), mode='nearest')
        #out = self.batchnorm(out)
        return out

In [22]:
spl = Splitted_Regression()
inp_seg, inp_reg, _,_ = Dataloaders_ver3['Prediction'].dataset[0:100]

In [23]:
re = spl(inp_reg)

torch.Size([100, 7, 100, 100])


Regression 결과 Bilinear Interpolation 함수 추가 -> Nearest Neighborhood로 수정해서 추가완료
## Regression 결과 Fully Convolutional Layer에 조합할 연산 추가, 이후 다시 Convolutional Layer 추가

In [ ]:
class UrbanGreenSegmentation(pl.LightningModule):
    def __init__(self, rotate_training_data : bool = False, train_ratio : float = 0.8, patch_size : int = 100, batch_size : int = 4, region:str = 'N12'):
        super(UrbanGreenSegmentation, self).__init__()
        raw_data_array, OHE_target_array, raw_target_array = prepare_raw_files(region)
        self.batch_size = batch_size
        self.Datasets = {
            'Train' : TrainDataset2(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, rotate = rotate_training_data, train_ratio = train_ratio),
            'Validation' : TrainDataset2(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, is_validating = True, rotate = rotate_training_data, train_ratio = train_ratio),
            'Prediction' : TrainDataset2(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, is_evaluating = True, train_ratio = train_ratio)
        }

        self.Dataloaders = {
            'Train' : DataLoader(self.Datasets['Train'], batch_size=batch_size),
            'Validation' : DataLoader(self.Datasets['Validation'], batch_size=batch_size),
            'Prediction' : DataLoader(self.Datasets['Prediction'], batch_size=batch_size)
        }
        
        

        self.unet = UNet()
        self.regression = Splitted_Regression()
        
        self.fc1 = nn.Conv2d(in_channels=32, out_channels=7)
        self.bn1 = nn.BatchNorm2d(7)
        self.fc2 = nn.Conv2d(in_channels=14, out_channels=7)
        self.softmax = nn.Softmax2d()

    def forward(self, x_seg, x_reg):
        x_reg = self.regression(x_reg)
        x_seg = self.unet(x_seg)
        x_seg = self.fc1(x_seg)
        x_seg = self.bn1(x_seg)
        x_seg = torch.cat((x_reg, x_seg), dim=1)
        x_seg = self.fc2(x_seg)
        x_seg = self.softmax(x_seg)
        return x_seg


    def training_step(self, batch, batch_idx):
        x_seg, x_reg, y_seg, _ = batch
        y_hat = self(x_seg, x_reg)
        #손실함수 추가하기
        pass

    def validation_step(self, batch, batch_idx):
        pass

    def predict_step(self, batch, batch_idx):
        pass

    def configure_optimizers(self):
        train_optimizer = torch.optim.Adam(self.parameters(), lr=0.02)
        train_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(train_optimizer, T_max=10)
        return [train_optimizer], [train_scheduler]

    def train_dataloader(self):
        return DataLoader(self.Datasets['Train'], batch_size = self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.Datasets['Validation'], batch_size = self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.Datasets['Prediction'], batch_size = self.batch_size)

In [29]:
class UrbanGreenSegmentation(nn.Module):
    def __init__(self, rotate_training_data : bool = False, train_ratio : float = 0.8, patch_size : int = 100, batch_size : int = 4, region:str = 'N12'):
        super(UrbanGreenSegmentation, self).__init__()
        '''raw_data_array, OHE_target_array, raw_target_array = prepare_raw_files(region)
        self.batch_size = batch_size
        self.Datasets = {
            'Train' : TrainDataset2(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, rotate = rotate_training_data, train_ratio = train_ratio),
            'Validation' : TrainDataset2(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, is_validating = True, rotate = rotate_training_data, train_ratio = train_ratio),
            'Prediction' : TrainDataset2(raw_data_array, OHE_target_array, raw_target_array, patch_size = patch_size, is_evaluating = True, train_ratio = train_ratio)
        }

        self.Dataloaders = {
            'Train' : DataLoader(self.Datasets['Train'], batch_size=batch_size),
            'Validation' : DataLoader(self.Datasets['Validation'], batch_size=batch_size),
            'Prediction' : DataLoader(self.Datasets['Prediction'], batch_size=batch_size)
        }'''
        
        

        self.unet = UNet()
        self.regression = Splitted_Regression()
        self.regression.to(device)
        
        self.fc1 = nn.Conv2d(in_channels=64, out_channels=7, kernel_size=1)
        self.bn1 = nn.BatchNorm2d(7)
        self.fc2 = nn.Conv2d(in_channels=14, out_channels=7, kernel_size=1)
        self.softmax = nn.Softmax2d()

    def forward(self, x_seg, x_reg):
        x_reg = self.regression(x_reg)
        print(x_reg.type())
        x_seg = self.unet(x_seg)
        x_seg = self.fc1(x_seg)
        x_seg = self.bn1(x_seg)
        print(x_seg.type())
        x_seg = torch.cat((x_reg, x_seg), dim=1)
        x_seg = self.fc2(x_seg)
        x_seg = self.softmax(x_seg)
        return x_seg

In [30]:
a, b, c, d = Dataloaders_ver3['Train'].dataset[0:2]

In [31]:
a = a.to(device)
b = b.to(device)

In [34]:
from torchsummary import summary
model_for_summary = UrbanGreenSegmentation()
model_for_summary.to(device)
summary(model_for_summary, [(3,284,284),(100,6,10,10)], batch_size=10)

torch.cuda.FloatTensor
torch.cuda.FloatTensor
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [10, 1, 10, 10]               7
       BatchNorm2d-2            [10, 1, 10, 10]               2
              ReLU-3            [10, 1, 10, 10]               0
            Linear-4                  [10, 256]          25,600
       BatchNorm1d-5                  [10, 256]             512
              ReLU-6                  [10, 256]               0
            Linear-7                  [10, 256]          65,536
       BatchNorm1d-8                  [10, 256]             512
              ReLU-9                  [10, 256]               0
           Linear-10                  [10, 256]          65,536
      BatchNorm1d-11                  [10, 256]             512
             ReLU-12                  [10, 256]               0
           Linear-13                  [10, 256]          

/home/bcyoon/anaconda3/envs/tt/lib/python3.7/site-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


TypeError: can't multiply sequence by non-int of type 'tuple'

In [35]:
model_for_summary(a,b)

torch.cuda.FloatTensor
torch.cuda.FloatTensor


tensor([[[[0.1420, 0.1660, 0.1452,  ..., 0.1174, 0.1625, 0.1482],
          [0.1239, 0.1116, 0.0998,  ..., 0.1333, 0.1255, 0.1157],
          [0.1752, 0.1546, 0.1158,  ..., 0.1176, 0.1067, 0.1264],
          ...,
          [0.1309, 0.1517, 0.1281,  ..., 0.0988, 0.1398, 0.0801],
          [0.1450, 0.1229, 0.1254,  ..., 0.1157, 0.0833, 0.0706],
          [0.1264, 0.1125, 0.1267,  ..., 0.1406, 0.1072, 0.1006]],

         [[0.1605, 0.1234, 0.1252,  ..., 0.1750, 0.1209, 0.1418],
          [0.0945, 0.1206, 0.0627,  ..., 0.1412, 0.1293, 0.1809],
          [0.1225, 0.1183, 0.1144,  ..., 0.1452, 0.1472, 0.1592],
          ...,
          [0.1315, 0.1069, 0.1518,  ..., 0.1138, 0.0860, 0.1184],
          [0.1343, 0.1277, 0.1081,  ..., 0.0797, 0.0932, 0.1229],
          [0.1243, 0.1622, 0.1132,  ..., 0.0795, 0.1273, 0.0819]],

         [[0.0764, 0.1231, 0.1862,  ..., 0.1463, 0.1234, 0.1330],
          [0.1508, 0.1051, 0.1295,  ..., 0.1199, 0.1230, 0.1386],
          [0.1211, 0.1799, 0.2477,  ..., 0